In [1]:
import random

In [2]:
from firebase_admin import initialize_app, credentials, db

In [4]:
cred = credentials.Certificate('creds.json')

In [5]:
app = initialize_app(cred, {'databaseURL': "https://calzan-default-rtdb.europe-west1.firebasedatabase.app/"})

In [11]:
H = "Holz"
L = "Lehm"
G = "Getreide"
W = "Wolle"
E = "Erz"
X = ""

class Game:
    
    resources = [H, L, G, W, E]
    
    def __init__(self, credentials):
        
        self.turn_ref = db.reference("/turn")
        self.board_ref = db.reference("/board")
        self.stack_ref = db.reference("/stack")
        self.player_refs = {a: db.reference('/players/' + s) for a, s in credentials.items()}
        self.player_count = len(credentials)
        self.resource_order = {r: i for i, r in enumerate(self.resources)}
        
        
    def set_board(self, board):
        
        self.board_ref.set(board)
        
    
    def initialize(self):
        
        for number, name in enumerate(self.player_refs):
            self.player_refs[name].set({
                "name": name,
                "number": number,
                "resources": [],
                "cards": [],
            })
        
        stack = ["Ritter"] * 14 + ["Monopol", "Strassenbau", "Erfindung"] * 2
        stack += ["Bibliothek", "Marktplatz", "Rathaus", "Kirche", "Universität"]
        random.shuffle(stack)
        self.stack_ref.set(stack)
        
        names = list(self.player_refs)
        first = random.randint(0, self.player_count - 1)
        self.turn_ref.set({'names': names, 'currentTurn': first, 'maxKnightCount': 2})
        self.set_turn(first, None)
        
    
    def set_turn(self, player_num, roll):

        self.current = player_num
        turn = self.turn_ref.get()
        turn['currentTurn'] = player_num
        turn['roll'] = roll
        self.turn_ref.set(turn)
        
        
    def set_longest_road(self, player_num, roads):
        
        turn = self.turn_ref.get()
        turn['longestRoadPlayer'] = player_num
        turn['longestRoadCount'] = roads
        self.turn_ref.set(turn)
        
        
    def next_turn(self, roll, step=1):
        
        next_player = (self.current + step) % self.player_count
        self.set_turn(next_player, roll)
        
        
    def next_roll(self):
        
        roll = random.randint(1, 6) + random.randint(1, 6)
        self.next_turn(roll)
        
        
    def update_player(self, player_num, gains=None, losses=None, gained_cards=None, lost_cards=None):
        
        player = self.player_refs[player_num].get()
        
        if losses is not None:
            for resource in losses:
                player['resources'].remove(resource)
        if gains is not None:
            player['resources'] += gains
        player['resources'] = sorted(player['resources'], key=lambda r: self.resource_order[r])
        
        if lost_cards is not None:
            for card in lost_cards:
                player['cards'].remove(card)
        if gained_cards is not None:
            player['cards'] += gained_cards
            if "Ritter (gespielt)" in gained_cards:
                knight_count = player['cards'].count("Ritter (gespielt)")
                turn = self.turn_ref.get()
                if knight_count > turn['maxKnightPlayer']:
                    turn['maxKnightCount'] = knight_count
                    turn['maxKnightPlayer'] = player_num
                    self.turn_ref.set(turn)
            
        self.player_refs[player_num].set(player)
            
            
    def trade_resources(self, giver_num, taker_num, give, take):
        
        self.update_player(giver_num, gains=take, losses=give)
        self.update_player(taker_num, gains=give, losses=take)
        
        
    def steal_resource(self, dest_num, orig_num):
        
        orig = self.player_refs[orig_num].get()
        if len(orig['resources']) == 0:
            return
        resource = random.choice(orig['resources'])
        self.update_player(orig_num, losses=[resource])
        self.update_player(dest_num, gains=[resource])
            
            
    def bandit(self):
        
        # TODO select which resources?
        for player_num, player_ref in self.player_refs:

            player = player_ref.get()
            over_count = len(player['resources']) - 7
            if over_count > 0:
                losses = random.sample(player['resources'], over_count)
                self.update_player(player_num, losses=losses)
        
        
    def buy(self, player_num, roads=0, towns=0, cities=0, cards=0):
        
        costs = [H, L] * roads + [H, L, G, W] * towns + [G, G, E, E, E] * cities + [G, W, E] * cards
        
        drawn_cards = None
        if cards > 0:
            stack = self.stack_ref.get()
            drawn_cards = stack[:cards]
            self.stack_ref.set(stack[cards:])
        
        self.update_player(player_num, losses=costs, gained_cards=drawn_cards)
        
        
    def play_card(self, player_num, card, param=None):
        
        gains = None
        gained_cards = None
        
        if card == 'Ritter':
            gained_cards = ["Ritter (gespielt)"]
                
        elif card == 'Strassenbau':
            # TODO: grant 2 streets?
            pass
            
        elif card == 'Monopol':
            
            gains = []
            for other_num, player_ref in enumerate(self.player_refs):
                if other_num == player_num:
                    continue
                other = self.player_refs[other_num].get()
                losses = [r for r in other['resources'] if r == param]
                gains += losses
                self.update_player(other_num, losses=losses)
                
        elif card == 'Erfindung':
            
            gains = params
            
        else:
            return
            
        self.update_player(player_num, gains=param, gained_cars=gained_cards, lost_cards=[card])

# Init

In [12]:
game = Game({'Ernie': 'kek', 'Bert': 'lel'})

In [16]:
game.set_board([    
"                                 x           ",
"                              + = +          ",
"                            /     |          ",
"                   e  + - +   G   + - +      ",
"                    %     |   4 /     $ w    ",
"              + - +   L   + - +   H   + - +  ",
"            /     |   2 /     |   6 /     |  ",
"          +   W   + - +   H   + - +   G   +  ",
"          |   9 /     |   5 /     |   9 /    ",
"       g  + - +   L   + - +   W   + - +      ",
"        %     |   8 /  *  |  12 /     $ x    ",
"      +   H   + - +   X   + - +   W   +      ",
"      |   3 /     |     /     |   4 /        ",
"      + - +   E   + - +   E   + - +          ",
"    /     |   5 /     |   8 /     |          ",
"  +   G   + - +   L   + - +   W   +          ",
"x $   3 /     |  10 /     |  10 %            ",
"  + - +   G   + - +   H   + - +  x           ",
"      |   6 /     |  11 /                    ",
"      + = +   E   + = +                      ",
"       h  |  11 /  l                         ",
"          + - +                              ",
"                                             "
])

In [14]:
tiles = [[H, W, W], [G, E, G, H], [H, L, X, E, G], [G, E, H, W], [L, W, L]]
rolls = [[6, 3, 8], [2, 4, 5, 10], [5, 9, 0, 6, 9], [10, 11, 3, 12], [8, 4, 11]]

# Play

In [13]:
game.initialize()

In [ ]:
game.next_turn(None, 1)

In [ ]:
game.next_turn(None, -1)

In [ ]:
game.roll_next()

In [ ]:
game.draw_card(0)

In [ ]:
game.play_card(0, "Ritter")

In [ ]:
game.steal_resource(0, 1)

In [ ]:
game.bandit()

In [ ]:
game.buy(0, roads=2)

In [ ]:
game.set_longest_road(0, 5)

In [ ]:
game.set_board([    
"                                 x           ",
"                              + = +          ",
"                            /     |          ",
"                   e  + - +   G   + - +      ",
"                    %     |   4 /     $ s    ",
"              + - +   L   + - +   H   + - +  ",
"            /     |   2 /     |   6 /     |  ",
"          +   W   + - +   H   + - +   G   +  ",
"          |   9 /     |   5 /     |   9 /    ",
"       w  + - +   L   + - +   W   + - +      ",
"        %     |   8 /  *  |  12 /     $ x    ",
"      +   H   + - +   X   + - +   W   +      ",
"      |   3 /     |     /     |   4 /        ",
"      + - +   E   + - +   E   + - +          ",
"    /     |   5 /     |   8 /     |          ",
"  +   G   + - +   L   + - +   W   +          ",
"x $   3 /     |  10 /     |  10 %            ",
"  + - +   G   + - +   H   + - +  x           ",
"      |   6 /     |  11 /                    ",
"      + = +   E   + = +                      ",
"       h  |  11 /  l                         ",
"          + - +                              ",
"                                             "
])